In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from dl.dataset.datamodes.npz import rectifiedrgb, debayeredrgb
from dl.config.label_map_helper import LabelMapHelper, LabelConversion

import pandas as pd
import os
from pathlib import Path
import json
from tqdm import tqdm

In [ ]:
output_dir_prod = '/mnt/sandbox1/alex.li/introspection/pmehta_2023_val_bestmodel/halo_rgb_stereo_test_v6_0'
output_dir_safety = '/mnt/sandbox1/alex.li/introspection/pmehta_2023_val_bestmodel/halo_humans_on_path_v3'
# output_dir_prod = '/mnt/sandbox1/alex.li/results/17902/halo_rgb_stereo_test_v6_0'
# output_dir_safety = '/mnt/sandbox1/alex.li/results/17902/halo_humans_on_path_v3'

In [ ]:
with open(output_dir_prod + "/results.txt", 'r') as f:
    print(''.join(f.readlines()))

In [ ]:
with open(output_dir_safety + "/results.txt", 'r') as f:
    print(''.join(f.readlines()))

In [ ]:
out_df = pd.read_csv(output_dir_prod + "/output.csv")

In [ ]:
prod_dset = '/data2/jupiter/datasets/halo_rgb_stereo_test_v6_0/'
master_df = pd.read_csv(prod_dset + "master_annotations.csv")

In [ ]:
fp_df = out_df[out_df['state'] == 'false_positive']
row = master_df[master_df['id'] == '64dea8faae8b0f37b46e05f7'].iloc[0]
# /mnt/sandbox1/alex.li/results/pmehta_2023_val_bestmodel/halo_rgb_stereo_test_v6_0/human_false_positive/64de93970bf522829d4cd6d8_T13_T15.png

In [ ]:
rec_artifacts = rec.get_artifacts(row)
label_converter = LabelConversion(label_map_helper)
rec_label = label_converter.convert_label_for_driveable_terrain(rec_artifacts['label'], json.loads(row['label_map']))


In [ ]:
import os
import random
dataset_path = '/mnt/sandbox1/alex.li/introspection/pmehta_2023_val_bestmodel/halo_rgb_stereo_test_v6_0/'
files = os.listdir(dataset_path + 'false_positive/')
os.mkdir(dataset_path + 'fp_subset_2/')
random.shuffle(files)
for f in files[:50]:
    os.symlink(dataset_path + 'false_positive/' + f, dataset_path + 'fp_subset_2/' + f)

In [ ]:
import os
import random
dataset_path = '/mnt/sandbox1/alex.li/introspection/pmehta_2023_val_bestmodel/halo_humans_on_path_v3/'
files = os.listdir(dataset_path + 'human_false_negative/')
os.mkdir(dataset_path + 'fn_subset/')
random.shuffle(files)
for f in files[:50]:
    os.symlink(dataset_path + 'human_false_negative/' + f, dataset_path + 'fn_subset/' + f)

In [ ]:
label_is_cutoff = []
label_is_humanonly = []
label_file = "/home/alex.li/git/JupiterCVML/europa/base/src/europa/dl/config/label_maps/eight_class_train_dust_light_as_sky_birds_as_driveable.csv"
label_map_helper = LabelMapHelper(label_file)
rec = rectifiedrgb.RectifiedRGBNPZ(prod_dset)
deb = debayeredrgb.DebayeredRGBNPZ(prod_dset)
label_converter = LabelConversion(label_map_helper)

for _, row in tqdm(master_df.iterrows(), total=len(master_df)):
    rec_artifacts = rec.get_artifacts(row)
    rec_label = label_converter.convert_label_for_driveable_terrain(rec_artifacts['label'], json.loads(row['label_map']))

    # for id in tqdm(master_df['id']):
    #     row = master_df[master_df['id'] == id].iloc[0]
    # deb_artifacts = deb.get_artifacts(row)
    # deb_label = label_converter.convert_label_for_driveable_terrain(deb_artifacts['label'], json.loads(row['label_map']))
    # rec_human = np.sum(rec_label == 5)
    # center_human = np.sum(rec_label[15:-15,15:-15] == 5)
    # edge_human = rec_human - center_human
    # deb_human = np.sum(deb_label == 5)
    # # Human is on edge only and more present in the debayered image
    # if edge_human > center_human and deb_human > rec_human * 8:
    #     #occluded human
    #     label_is_cutoff.append(True)
    # else:
    #     label_is_cutoff.append(False)
    label_is_humanonly.append(set(np.unique(rec_label)) <= set({5, 255}))

In [ ]:
sum(label_is_humanonly)

In [ ]:
master_df_full_label = master_df[~np.array(label_is_humanonly)]
master_df_full_label.to_csv(prod_dset + "master_annotations_full_label.csv")

In [ ]:
master_df_full_label = pd.read_csv(prod_dset + "master_annotations_full_label.csv")

In [ ]:
all_uid = master_df['unique_id']
other_uid = master_df_full_label['unique_id']
diff_id = set(all_uid) - set(other_uid)

In [ ]:
humanonly_df = master_df[master_df['unique_id'].isin(diff_id)]

In [ ]:
len(humanonly_df)

In [ ]:
me = pd.read_csv(prod_dset + "master_annotations_human_label.csv")
pawan = pd.read_csv("/home/alex.li/logs/only_humans_labeled_v2.csv")

In [ ]:
me['annotation_pixelwise_0_labeled_objects']

In [ ]:
pawan['id']

In [ ]:
pawan_uid = pawan['id']
me_uid = me['id']
in_pawan_only = list(set(pawan_uid) - set(me_uid))
in_me_only = list(set(me_uid) - set(pawan_uid))

print(in_pawan_only[:4])
print(in_me_only[:4])

In [ ]:
print(len(in_me_only))
print(len(in_pawan_only))